import libraries

In [1]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

define important variables

In [2]:
PHONE_NUMBER = "123A456B789C*0#D"


dtmf_table_freq = {
    (1209, 697): "1",
    (1336, 697): "2",
    (1477, 697): "3",
    (1633, 697): "A",
    (1209, 770): "4",
    (1336, 770): "5",
    (1477, 770): "6",
    (1633, 770): "B",
    (1209, 852): "7",
    (1336, 852): "8",
    (1477, 852): "9",
    (1633, 852): "C",
    (1209, 941): "*",
    (1336, 941): "0",
    (1477, 941): "#",
    (1633, 941): "D",
}

indexes = {
    '1' : 0,
    '2' : 1,
    '3' : 2,
    '4' : 3,
    '5' : 4,
    '6' : 5,
    '7' : 6,
    '8' : 7,
    '9' : 8,
    '0' : 9,
    'A' : 10,
    'B' : 11,
    'C' : 12,
    'D' : 13,
    '#' : 14,
    '*' : 15,
}

Generate DTMF Signals based on digit

In [3]:
def waveform(phone_num, duration=100, gap=50, sampling_rate=8000, amplitude=1, graphic=False, mean=0, standard_deviation=0):
    '''Returns numpy array for the values of the corresponding waveform.
    There's also a 'graphic' option to include a graph.'''
    dtmf_table = {
        "1": [1209, 697],
        "2": [1336, 697],
        "3": [1477, 697],
        "A": [1633, 697],
        "4": [1209, 770],
        "5": [1336, 770],
        "6": [1477, 770],
        "B": [1633, 770],
        "7": [1209, 852],
        "8": [1336, 852],
        "9": [1477, 852],
        "C": [1633, 852],
        "*": [1209, 941],
        "0": [1336, 941],
        "#": [1477, 941],
        "D": [1633, 941],
    }
    phone_num = phone_num.upper()
    for digit in phone_num:
        if digit not in dtmf_table:
            raise ValueError('The only allowed digits are 0-9, A-D, #, and *')

    gap_amount_of_samples = gap * sampling_rate // 1000
    gap_values = np.zeros(gap_amount_of_samples)

    first = True
    ans = np.array([])
    amount_of_samples = (duration/1000)*sampling_rate
    samples = np.arange(0, duration/1000, (duration/1000)/amount_of_samples)
    for digit in phone_num:
        if not first:
            ans = np.append(ans, gap_values)
        else:
            first = False
        values = (amplitude/2) * (
                np.sin(samples * 2 * np.pi * dtmf_table[digit][0]) +
                np.sin(samples * 2 * np.pi * dtmf_table[digit][1])
        )
        ans = np.append(ans, values)
    # plotting
    if graphic:
        plt.title("Line graph")
        plt.xlabel("X axis")
        plt.ylabel("Y axis")
        plt.plot(np.arange(0, (len(phone_num)-1)*gap_amount_of_samples+len(phone_num)*amount_of_samples), ans, color="green")
        plt.show()
    ans = ans + np.random.normal(mean, standard_deviation, len(ans))

    return ans

Use Welch's algorithm to graph periodogram

In [4]:
def welch(waveform, fs):
    f, Pwelch_spec = signal.welch(waveform, fs, scaling='spectrum')
    #plt.semilogy(f, Pwelch_spec)
    #plt.xlabel('frequency [Hz]')
    #plt.ylabel('PSD')
    #plt.grid()
    #plt.show()
    return [f, Pwelch_spec]

Function to closest frequency

In [5]:
def closest(lst, K):
    return lst[min(range(len(lst)), key=lambda i: abs(lst[i] - K))]

Finds digit based on clean periodogram

In [6]:
def frequencies(f, pwelch_spec):
    low_dict = {}
    high_dict = {}
    for i in range(len(f)):
      if f[i] < 1050:
        low_dict[pwelch_spec[i]] = f[i]
      elif f[i] < 1800:
        high_dict[pwelch_spec[i]] = f[i]

    low_range = list(low_dict.keys())
    high_range = list(high_dict.keys())
    low_freq = low_dict[max(low_range)]
    high_freq = high_dict[max(high_range)]
    lows = [697, 770, 852, 941]
    highs = [1209, 1336, 1477, 1633]
    return dtmf_table_freq[(closest(highs, high_freq), closest(lows, low_freq))]

Adds noise and attempts to find digit based on periodogram. Uses a confusion matrix to map out accuracy based on standard deviation.

In [7]:
for standard_dev in range(0,30,3):
  standard_dev = standard_dev/10
  confusion_matrix = np.zeros(256).reshape(16,16)
  names = [_ for _ in PHONE_NUMBER]
  for digit in PHONE_NUMBER:
    i = 1
    while i<=100:
       temp = welch(waveform(digit, standard_deviation=standard_dev), 8000)
       guess = frequencies(temp[0], temp[1])
       confusion_matrix[indexes[digit]][indexes[guess]] += 1
       i += 1
  df = pd.DataFrame(confusion_matrix, index=names, columns=names)
  df.to_csv('standard_dev' + str(standard_dev) + '.csv', index=True, header=True, sep=',')
  print(df)

       1      2      3      A      4      5      6      B      7      8  \
1  100.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2    0.0  100.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3    0.0    0.0  100.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
A    0.0    0.0    0.0  100.0    0.0    0.0    0.0    0.0    0.0    0.0   
4    0.0    0.0    0.0    0.0  100.0    0.0    0.0    0.0    0.0    0.0   
5    0.0    0.0    0.0    0.0    0.0  100.0    0.0    0.0    0.0    0.0   
6    0.0    0.0    0.0    0.0    0.0    0.0  100.0    0.0    0.0    0.0   
B    0.0    0.0    0.0    0.0    0.0    0.0    0.0  100.0    0.0    0.0   
7    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  100.0    0.0   
8    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  100.0   
9    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
C    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
*    0.0    0.0    0.0   